### 介绍两种句法结构sentence structure：Consistency and Dependency
- Consistency = phrase structure grammar = context-free grammars(CFGs)就是为句子设定具体的语法，用语法限制和分析句子。<br>
最简单的可以用三元式（包括，变量，常量和运算符）表示语法。A -> B + B; B -> A|c;其中大写字母表示变量，小写表示constant。那么这句语法就表示任意多个constant相加。教授讲的那个，大致意思相同，但是具体的真没学过。
- dependency structure给出了一个句子中，各个单词之间的依赖关系。然后用箭头描述这种依赖。<br>但是这种方法存在ambiguity：
比如scientists study whales from the space.你并不知道whale是否来自太空。 <br>
另外对于一个状词短语，你很难确定，他所描述（依赖）的对象是什么。比如：The board approved [its acquisition] [by Royal Trustco Ltd.] [of Toronto] [for $27 a share] [at its monthly meeting]。也就是说，不存在一个特定的规则，比如说相邻依赖，来规范这些PPs的依赖关系<br>
依赖数和CatAlan数的关系是在句子遵守一般规则（不交叉依赖）的情况下才适用的，在这种情况下，可以把依赖双方想象成括号，这样就好理解了。前面这个同时也是在限制为1对1的情况，也就是说真实情况应该是对于CatAlan数的。<br>
CatAlan numbers is an exponentially growing series, which arises in many tree-like contexts

#### treebank
树库是指存储句子信息的语料库，因为他用树结构存储句子信息，所有叫做treebank。根据所描述结构的不同，树库大致可分为两类：短语结构树库，和依存结构树库。<br>
短语结构树库，一般采用句子的结构成分描述句子的结构，短语结构树包含了规则与依存双重信息，他可以用于提取短语，他的目的是分析句子产生的过程。<br>
依存结构树库，是根据句子的依存结构而建立的树库。依存结构描述了句子中词与词之间直接的句法关系，相应的树也称为依存树。与短语结构树库不同，他的目标并不是探究句子是怎么产生的，而是研究已产生的句子之间的依存关系<br>
课程里提到的http://universaldependencies.org/ 就是依存结构树库

grammar vs treebank:
Starting off ,building a treebank seems a lot slower and less useful than building a grammar<br>
but a treebank gives us many thing:
* Reusability of the labor, 语料库都懂的
* Broad coverage, not just a few intuitions
* Frequencies and distributional information
* a way to evaluate systems

### 什么是dependency grammar, dependency structure
依赖语法假设一个句子的语法结构由单词之间的依赖构成（每次翻译完都觉得会遗漏信息，先汗一个）<br>
Dependecy syntax postulates(o±o) that syntactic structure consists of relations between lexical items, normally binary asymmetric relations("arrows") called dependencies<br>
The arrow connects a head(goernor, superior, regent) with a dependent(modifier, inferior, subordinate)<br>
相对于有严谨的语法结构，依存语法能更好的使用于自由度比较高的一些语言，比如中文，俄语。但其实加上口语这种语言，应该也是可以的吧。
usually add a fake Root so every word is a dependent of precisely 1 other node

![dependency syntax](https://raw.githubusercontent.com/HuangYiran/Natural_Language_Processing_with_Deep_Learning/master/dependency%20syntax.png)

#### 我们如何确定两个单词之间的依赖关系
一般我们可以考虑下面这几个信息：
* bilexical affinities [discussion -> issues] is plausible 这里是指语义上的分析吗？？
* dependency distance 大部分存在依赖的单词组之间的距离，是比较小的。
* intervening material 大部分存在依赖的单词组之间不会有动词或标点
* valency of heads 不同的单词由不同的依赖习惯

#### 应该怎么进行dependency parsing
A sentence is parsed by choosing for each word what other word(including ROOT) is it a dependent of .<br>
Usually we will add some constrains to garanty that we can get a tree after the parsing like:
- only one word is dependent of ROOT
- don't want cycles A -> B, B -> A

如果有交叉怎么办？？

### methods of dependency parsing
1. Dynamic programming<br>
Eisner(1996) gives a clever algorithm with complexity O$(n^3)$, by producing parse items with heads at the ends rather than in the middle
2. Graph algorithms<br>
You create a Minimum Spanning Tree for a sentence<br>
McDonald(2005)'s MSTParser scores dependencies independently using an ML classifier(he uses MIRA, for online learning, but it can be something else)
3. Constraint Satisfaction<br>
Edges are eliminated that don't satisfy hard constraints. Karlsson(1990), etc
4. "Transition-based parsing" or "deterministic dependency parsing"<br>
Greedy choice of attachments guided by good machine learning classifiers<br>
MaltParser(2008). Has proven highly effective

### Greedy transition-based parsing
The parse does a sequence of bottom up actions. Roughly like "shift" or "reduce" in a shift -reduce parse, but the "reduce" actions are specialized to create dependencies with head on left or right. 记得当时学shift-reduce parse还是在本科编译系统的课上（结果是60年代的产物）先汗一个<br>
The parser has:
* a stack $\sigma$, written with top to the right, witch starts with the ROOT symbol
* a buffer $\beta$, written with top to the left, which starts with the input sentence
* a set of dependency arcs A, which starts off empty
* a set of actions

action包括有三个：
* shift:  $\sigma, w_i|\ \beta,\ A\ ->\ \sigma|W_i,\ \beta,\ A $<br>
w表示buffer中的单词，shift的意思就是把buffer顶部的一个单词拿到stack中。
* Left_Arc: $\sigma|w_i|w_j,\ \beta,\ A\ ->\ \sigma|w_j,\ \beta, A\cup\{r(w_j,w_i\}$<br>
把stack中第二个单词拿掉，与stack中首个单词组成依赖，放到依赖集中，w_j为head
* right_Arc: $\sigma|w_i|w_j,\ \beta,\ A\ ->\ \sigma|w_j,\ \beta, A\cup\{r(w_i,w_j\}$<br>
与Left_Arc操作相同，不过依赖的方向倒了。另外注意，stack中首个单词并没有拿掉，拿掉的只是第二个单词。
*这是最简单的parse，他只能标出两个单词之间有没有依赖，而是什么依赖，则无能为力。扩展的方法就是扩展action的数量，为每一种依赖关系分别定义两个actions，即Left_Arc和right_Arc。

当$\beta$为空时，终止parsing。

#### 那么应该如何选择，使用那个action呢？MaltParser(2005)使用的方法如下：
Each actioin is predicted by a discriminative clasifier(often SVM, can be perceptron, maxent classifier) over each legal move 
* Max of 3 untyped choices; max of |R|*2 + 1 when typed
* Features: top of stack word, POS; first in buffer word, POS, etc

#### 使用NN进行action选择训练
具体的网络结构就不说了，输出层使用softmax进行action选择。输入层feature可以是stack和buffer的前几个单词及他们的类型。我们使用one-hot来表示每个位置的选词及其类型，由于单词和类型的数量众多，feature的数量级可以达到$10^6~10^7$(比如可以合理的假设，有1000个单词，100中类型)。同时这个网络好存在以下这些问题：
- sparse，feature的数量，可能比你的treebank还大，很多时候，一些组合只会出现一到两次
- incomplete，会出现一些奇奇怪怪的组合，你压根就没有见过的，所以你也不会有他们的feature
- expensive computation

#### 改进方法：C & M2014
- 使用distribute representation表示单词，part-of-speech(POS)和dependency label
- concatenate这些vector，组成输入。